# Introducción a Polars

**Polars** es un motor de análisis de datos moderno, escrito en Rust y diseñado para ser extremadamente rápido y eficiente. A diferencia de Pandas, que puede ser lento con datasets grandes, Polars está optimizado para:

* **Velocidad:** Implementación en Rust con evaluación lazy
* **Memoria:** Uso eficiente de RAM con Arrow como backend
* **API intuitiva:** Expresiones composables similar a SQL
* **Escalabilidad:** Diseñado para Big Data

En este notebook aprenderemos a trabajar con Polars desde cero.

In [ ]:
import polars as pl
import pandas as pd
import numpy as np

print(f"Polars versión: {pl.__version__}")
print(f"Polars info: {pl.show_versions()}")

## ¿Por qué Polars?

### Comparativa: Pandas vs Polars

| Aspecto | Pandas | Polars |
| :--- | :--- | :--- |
| **Lenguaje** | Python | Rust + Python binding |
| **Backend** | NumPy | Arrow |
| **Velocidad** | Moderada | ⚡ Muy rápida |
| **Memoria** | Eficiente | 🎯 Óptima |
| **Lazy eval.** | No | Sí (opcional) |
| **API** | Imperativos | Expresiones |
| **Escalabilidad** | En-memoria | In-memory + distribuido |

### Benchmarks Reales

En operaciones típicas de análisis de datos, Polars puede ser **5-10x más rápido** que Pandas.

In [ ]:
# Benchmark: Creación de DataFrame
import time

n_rows = 1_000_000

# Benchmark Pandas
start = time.time()
df_pandas = pd.DataFrame({
    'id': range(n_rows),
    'valor': np.random.rand(n_rows)
})
tiempo_pandas = time.time() - start

# Benchmark Polars
start = time.time()
df_polars = pl.DataFrame({
    'id': range(n_rows),
    'valor': np.random.rand(n_rows)
})
tiempo_polars = time.time() - start

print(f"Pandas: {tiempo_pandas:.4f}s")
print(f"Polars: {tiempo_polars:.4f}s")
print(f"Polars es {tiempo_pandas/tiempo_polars:.1f}x más rápido")

## DataFrames en Polars

Un DataFrame en Polars es similar al de Pandas, pero con una API más eficiente y expresiva.

In [ ]:
# Crear DataFrame desde diccionario
df = pl.DataFrame({
    'id': [1, 2, 3, 4, 5],
    'nombre': ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve'],
    'edad': [25, 30, 35, 28, 32],
    'salario': [50000.0, 60000.0, 75000.0, 55000.0, 70000.0]
})

print(df)

In [ ]:
# Inspeccionar el DataFrame
print(f"Shape: {df.shape}")
print(f"\nColumns: {df.columns}")
print(f"\nSchema:\n{df.schema}")
print(f"\nInfo:\n{df.info()}")

In [ ]:
# Head y tail
print("Head:")
print(df.head(2))
print("\nTail:")
print(df.tail(2))

In [ ]:
# Describe (estadísticas)
print(df.describe())

In [ ]:
# Leer CSV con Polars
df_csv = pl.read_csv('datos_ejemplo.csv')
print(f"DataFrame desde CSV:\n{df_csv}")

In [ ]:
# Leer Parquet
df_parquet = pl.read_parquet('tabla_ejemplo.parquet')
print(f"DataFrame desde Parquet:\n{df_parquet}")

## Selecciones y Filtros con Expresiones

Polars utiliza un sistema de **expresiones** que permite operaciones claras y composables.

In [ ]:
# Select: elegir columnas
df_select = df.select(['nombre', 'edad'])
print("Select columns:")
print(df_select)

In [ ]:
# Filter: filtrar filas
df_filtered = df.filter(pl.col('edad') > 28)
print("Personas con edad > 28:")
print(df_filtered)

In [ ]:
# Expresiones combinadas
df_complex = df.filter(
    (pl.col('edad') > 25) & (pl.col('salario') >= 60000)
)
print("Edad > 25 AND Salario >= 60000:")
print(df_complex)

## Transformaciones

Agregar, modificar o eliminar columnas con `with_columns()`.

In [ ]:
# Agregar columnas
df_transformed = df.with_columns(
    (pl.col('salario') * 1.1).alias('salario_incrementado'),
    (pl.col('edad') / 10).alias('edad_decada')
)
print("Con columnas calculadas:")
print(df_transformed)

In [ ]:
# Renombrar columnas
df_renamed = df.rename({
    'nombre': 'employee_name',
    'edad': 'employee_age'
})
print(df_renamed)

In [ ]:
# Drop (eliminar) columnas
df_dropped = df.drop('salario')
print("Sin columna salario:")
print(df_dropped)

## Groupby y Agregaciones

Agrupar datos y aplicar funciones de agregación de forma eficiente.

In [ ]:
# Crear datos de ejemplo con categorías
df_sales = pl.DataFrame({
    'departamento': ['Ventas', 'IT', 'Ventas', 'IT', 'HR', 'Ventas', 'IT', 'HR'],
    'empleado': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'],
    'salario': [50000, 80000, 55000, 85000, 45000, 60000, 90000, 48000]
})
print(df_sales)

In [ ]:
# Groupby con agregaciones
df_agg = df_sales.groupby('departamento').agg(
    pl.col('salario').mean().alias('salario_promedio'),
    pl.col('salario').max().alias('salario_max'),
    pl.col('empleado').count().alias('cantidad')
)
print("Aggregations por departamento:")
print(df_agg)

In [ ]:
# Ordenar resultados
df_sorted = df_sales.sort('salario', descending=True)
print("Ordenado por salario (descendente):")
print(df_sorted)

## Lazy Evaluation

Polars soporta **evaluación lazy**, donde las operaciones se definen pero no se ejecutan hasta que se llama `.collect()`.

Esto permite que Polars optimice automáticamente las consultas.

In [ ]:
# Evaluación lazy
query = (df
    .lazy()
    .filter(pl.col('edad') > 27)
    .select(['nombre', 'salario'])
    .sort('salario', descending=True)
)

print(f"Query type: {type(query)}")
print("\nEjecutar query:")
result = query.collect()
print(result)

In [ ]:
# Ver el plan de ejecución optimizado
query_plan = (df
    .lazy()
    .filter(pl.col('edad') > 27)
    .select(['nombre', 'salario'])
    .sort('salario')
)

print("Query plan (optimizado):")
print(query_plan.explain())

## Conversiones Pandas ↔ Polars

Es fácil convertir entre Pandas y Polars.

In [ ]:
# Pandas → Polars
df_pandas = pd.DataFrame({
    'A': [1, 2, 3],
    'B': ['x', 'y', 'z']
})
df_polars = pl.from_pandas(df_pandas)
print(f"Pandas a Polars:\n{df_polars}")

In [ ]:
# Polars → Pandas
df_back_to_pandas = df_polars.to_pandas()
print(f"Polars a Pandas:\n{df_back_to_pandas}")
print(f"\nTipo: {type(df_back_to_pandas)}")

## Resumen: Polars vs Pandas

### Cuándo usar Polars:

✓ Datasets grandes (>1GB)

✓ Operaciones complejas que requieren optimización

✓ Lazy evaluation para consultas complejas

✓ Necesidad de máxima velocidad

### Cuándo usar Pandas:

✓ Datasets pequeños y medianos

✓ Compatibilidad con código existente

✓ Visualización rápida

✓ Ecosistema amplio de librerías

En el próximo notebook veremos **operaciones avanzadas** en Polars: Window functions, joins optimizados y lazy evaluation profunda.